# **TF-IDF로 키워드 추출하기**

### **데이터셋**
- **출처**: 네이버 뉴스
- **내용**: 
  - 1일(24시간 이내)간 올라온 8개 세부 카테고리의 뉴스 데이터.
  - 포함된 열: 
    - **뉴스 제목**
    - **뉴스 내용**

### **내용**
1. 데이터 불러오기 & 결측값 확인
2. 데이터 정제
3. 키워드 추출

---

**작성자 정보** <br>
Written by [**Yujin_nKim**](https://github.com/Yujin-nKim)

## **1. 데이터 불러오기 & 결측값 확인**

In [26]:
import pandas as pd

# CSV 파일 읽기
news_data = pd.read_csv("dataset_news_data_detailed.csv")

# 데이터 크기 출력
print(f"데이터 크기 | (뉴스 개수, 열 개수) : {news_data.shape}")

데이터 크기 | (뉴스 개수, 열 개수) : (7837, 4)


In [27]:
# Pandas 설정 변경 : 열 너비 제한 해제
pd.set_option("display.max_colwidth", None)

In [28]:
# 1. 결측값 개수 확인
print("결측값 개수:")
print(news_data.isnull().sum())

# 2. 결측값이 포함된 행 출력
print("\n결측값이 포함된 행:")
print(news_data[news_data.isnull().any(axis=1)])

결측값 개수:
카테고리      0
뉴스 제목    10
뉴스 내용    55
뉴스 링크     0
dtype: int64

결측값이 포함된 행:
        카테고리                                          뉴스 제목 뉴스 내용  \
98        금융                                     [표] 외국환율고시   NaN   
262       금융           [속보]비트코인, 트럼프 취임 직전 최고가 갱신…10만8943달러   NaN   
606       금융                 [속보] 리플 10% 이상 폭락, 2.91달러까지 추락   NaN   
615       금융                      [속보] 리플 8% 이상 급락, 3달러도 붕괴   NaN   
620       금융            [속보] 리플 낙폭 더 늘려…7% 급락, 3.01달러까지 떨어져   NaN   
623       금융                    [속보] 트럼프 코인 때문에 리플 3% 이상 급락   NaN   
635       금융                [속보] 트럼프 취임 앞두고 트럼프 밈코인 900% 폭등   NaN   
703       금융                                     [표] 외국환율고시   NaN   
714       금융                  [속보] 비트코인, 사상 최고가 경신…1억5720만원   NaN   
893       금융        [속보]트럼프 취임 코앞, 비트코인 5%↑ 10만5884달러까지 치솟아   NaN   
932       금융                                    [표] 외국환율고시표   NaN   
961       금융                                            NaN   NaN   
1548      증권    [속보]中

In [29]:
# 결측값 확인

# 1. 뉴스 제목이 결측값이고 뉴스 내용만 존재하는 데이터
news_title_missing = news_data[news_data["뉴스 제목"].isnull() & news_data["뉴스 내용"].notnull()]
print("뉴스 제목이 결측값이고 뉴스 내용만 존재하는 데이터:")
print(news_title_missing)  # 출력 결과: 없음

# 2. 뉴스 제목만 있는 경우 (결측값이 포함된 행의 뉴스 링크 출력)
print("\n뉴스 제목만 있고 뉴스 내용은 이미지로만 있는 경우 (링크):")
missing_content_links = news_data.loc[news_data.isnull().any(axis=1), "뉴스 링크"]
print(missing_content_links.head())  # 해당 링크를 통해 뉴스 상태 확인

뉴스 제목이 결측값이고 뉴스 내용만 존재하는 데이터:
Empty DataFrame
Columns: [카테고리, 뉴스 제목, 뉴스 내용, 뉴스 링크]
Index: []

뉴스 제목만 있고 뉴스 내용은 이미지로만 있는 경우 (링크):
98     https://n.news.naver.com/mnews/article/009/0005432001
262    https://n.news.naver.com/mnews/article/277/0005535560
606    https://n.news.naver.com/mnews/article/421/0008031264
615    https://n.news.naver.com/mnews/article/421/0008031248
620    https://n.news.naver.com/mnews/article/421/0008031229
Name: 뉴스 링크, dtype: object


### 데이터 처리 로직
1. **뉴스 제목만 있는 경우**  
   - 뉴스 제목만 데이터로 사용.

2. **뉴스 제목과 뉴스 내용이 모두 없는 경우**  
   - 해당 데이터를 결측값으로 처리.

In [30]:
# 뉴스 데이터 결측값 처리

# 1. 뉴스 제목과 뉴스 내용이 모두 결측값인 행 개수 확인
num_missing_rows = news_data[["뉴스 제목", "뉴스 내용"]].isnull().all(axis=1).sum()
print(f"제거 전: 뉴스 제목과 뉴스 내용이 모두 결측값인 행 개수 = {num_missing_rows}")

# 2. 결측값 제거 전 총 행 개수
num_rows_before = len(news_data)

# 3. 뉴스 제목과 뉴스 내용이 모두 결측값인 행 제거
df_cleaned = news_data.dropna(subset=["뉴스 제목", "뉴스 내용"], how="all")

# 4. 결측값 제거 후 총 행 개수
num_rows_after = len(df_cleaned)

# 5. NaN 값을 빈 문자열로 채움
df_cleaned.loc[:, "뉴스 제목"] = df_cleaned["뉴스 제목"].fillna("")
df_cleaned.loc[:, "뉴스 내용"] = df_cleaned["뉴스 내용"].fillna("")

# 6. 처리 결과 출력
print(f"제거 전 총 행 개수: {num_rows_before}")
print(f"제거 후 총 행 개수: {num_rows_after}")

제거 전: 뉴스 제목과 뉴스 내용이 모두 결측값인 행 개수 = 10
제거 전 총 행 개수: 7837
제거 후 총 행 개수: 7827


## **2. 데이터 정제**

<details>
  <summary>뉴스 제목 + 뉴스 내용 예시 (정제 전)</summary>
“우리 동창회 총무, 돈 관리 잘 하고 있나”…이 통장 만들면 서로 편하다는데	저비용으로 효과 커 효자상품\n신한銀, 3년 만에 재출시 나서\n저축은행도 상품 출시 채비\n\n\n[사진 = 연합뉴스]\n금융권이 신규 고객 유치의 수단으로 여러 사람이 함께 계좌를 관리할 수 있는 ‘모임통장’을 잇달아 내놓고 있다. 모임통장은 출시·관리에 큰 비용이 들지 않지만 많은 자금을 조달할 수 있어 효자 상품이다. 차별화를 위해 단체적금, 영수증 첨부 기능 등을 내놓는 곳도 등장했다.\n\n20일 금융권에 따르면 신한은행은 다음달 ‘SOL모임통장’을 출시한다. 신한은행은 2011년 모임통장 전용 앱 ‘김총무’를 출시했지만 저조한 이용률 등을 이유로 2022년 6월 서비스를 중단했다가 3년 만에 재출시한다.\n\n시중은행 모임통장 상품 중 최초로 적금통장 기능을 갖췄다. 모임 멤버들이 특정 목표를 위해 금액과 기간을 설정하고 다 같이 돈을 모을 수 있도록 했다. 신한은행 고객이 아니더라도 모바일 웹을 통해 모임통장에 참여할 수 있다. 과거 금융권에서 출시된 모임통장은 해당 금융사 이용자가 앱을 설치해야만 참여가 가능했다.\n\n거래내역 영수증 첨부 기능도 더해 통장에 들어오거나 나가는 돈의 액수와 사용처를 투명하게 관리할 수 있도록 했다. 참여 멤버들에게 모임통장의 거래 세부 내용을 속이는 방식으로 ‘곗돈 먹튀’가 발생하는 일을 막을 수 있다는 게 신한은행 측 설명이다. 모임통장에 모인 돈을 한동안 쓸 일이 없을 때 파킹통장에 넣어둬 이자를 더 받는 것도 가능하다.\n\nKB국민은행도 모임통장 서비스를 운영 중이다. 지난해 하반기 선보인 ‘KB모임금고’라는 파킹통장 서비스다. 모임통장 서비스에 가입한 고객이 여유자금을 보관할 수 있도록 하는 상품이며 최고금리는 연 2.0%다.\n\n우리은행은 지난해 11월 새 뱅킹 앱인 ‘뉴원뱅킹’ 출시와 함께 앱에 모임통장 기능을 신설했다. 거래 내역은 그대로 두고 모임장을 교체할 수 있으며, 하나의 모임에서 카드 여러 장을 발급받을 수 있도록 했다.\n\n하나은행에서도 ‘하나원큐’ 앱을 통해 모임통장 서비스 이용이 가능하다. 기존에 사용 중이던 통장에 모임 기능을 연결해 모임통장으로 쓸 수 있다. 납부일·미납사실 자동 알림 기능을 갖췄으며, 음식점이나 커피전문점 등에서 모임 전용 체크카드 이용 시 월 최대 1만원의 캐시백을 받을 수 있다. NH농협은행 또한 ‘올원뱅크’ 앱과 연계한 ‘NH모여라통장’을 운영 중이다.\n\n모임통장은 수시 입출금통장 형태가 대부분이다. 기본금리는 연 0.1% 수준이다. 카카오뱅크, 케이뱅크, 토스뱅크와 같은 인터넷전문은행도 다양한 상품을 내놓고 있다.\n\n저축은행도 조만간 모임통장 서비스에 뛰어들 예정이다. 저축은행중앙회는 모임통장 시스템 구축 절차를 밟고 있다.\n\n시중은행 관계자는 “은행은 모임통장으로 일반 예·적금 상품보다 많은 자금을 조달할 수 있다”면서 “모임에 참여하는 사람들이 은행의 다른 서비스를 이용하도록 유도할 수 있다는 것도 장점”이라고 설명했다. 
  
</details>

In [20]:
import re

# 2. 텍스트 정제 함수 정의
def text_clean(text):
    """
    텍스트 데이터를 정제하는 함수
    """
    text = re.sub(r'<[^>]*>', '', text)
    
    text = re.sub(r'[ㄱ-ㅎㅏ-ㅣ]+', '', text)
    
    text = re.sub(r'\[.*?\]|기자', '', text)
    
    text = text.replace('\n', ' ')
    
    return text

# 3. 뉴스 제목과 뉴스 내용 정제
df_cleaned["뉴스 제목 정제"] = df_cleaned["뉴스 제목"].apply(text_clean)
df_cleaned["뉴스 내용 정제"] = df_cleaned["뉴스 내용"].apply(text_clean)

# 4. 정제 결과 확인
print(df_cleaned[["뉴스 제목", "뉴스 제목 정제", "뉴스 내용", "뉴스 내용 정제"]].head())

# 5. 정제된 데이터를 CSV 파일로 저장
output_file = "cleaned_dataset_news_data_detailed.csv"
df_cleaned[["뉴스 제목 정제", "뉴스 내용 정제"]].to_csv(output_file, index=False)

print(f"정제된 데이터가 '{output_file}'에 저장되었습니다.")

/var/folders/7l/l6h4r0h1627dcgyc5bdvldtr0000gn/T/ipykernel_79605/181320719.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned["뉴스 제목 정제"] = df_cleaned["뉴스 제목"].apply(text_clean)


                                           뉴스 제목  \
0    “우리 동창회 총무, 돈 관리 잘 하고 있나”…이 통장 만들면 서로 편하다는데   
1      [이코노미 플러스]지난해 불법사금융 피해신고 급증…채권추심 4년만에 5배로   
2  [이코노미 플러스] 금융당국 저축은행 여러곳 '취약' 추가통보…적기시정조치 가능성   
3       [이코노미 플러스]공매도 통합 가이드라인 마련…내부통제기준 법인별 차등화   
4       [이코노미 플러스] “상호금융 주담대 부실 가능성…리스크 관리 강화해야”   

                                   뉴스 제목 정제  \
0   우리 동창회 총무 돈 관리 잘 하고 있나이 통장 만들면 서로 편하다는데   
1      이코노미 플러스지난해 불법사금융 피해신고 급증채권추심 년만에 배로   
2  이코노미 플러스 금융당국 저축은행 여러곳 취약 추가통보적기시정조치 가능성   
3     이코노미 플러스공매도 통합 가이드라인 마련내부통제기준 법인별 차등화   
4       이코노미 플러스 상호금융 주담대 부실 가능성리스크 관리 강화해야   

                                                                                                                                                                                                                                                                                                                                                                                                                    

/var/folders/7l/l6h4r0h1627dcgyc5bdvldtr0000gn/T/ipykernel_79605/181320719.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned["뉴스 내용 정제"] = df_cleaned["뉴스 내용"].apply(text_clean)


## **3. 키워드 추출하기**

In [31]:
data = pd.read_csv("cleaned_dataset_news_data_detailed.csv")

data.head()

,뉴스 제목 정제,뉴스 내용 정제
0,우리 동창회 총무 돈 관리 잘 하고 있나이 통장 만들면 서로 편하다는데,저비용으로 효과 커 효자상품 신한銀 년 만에 재출시 나서 저축은행도 상품 출시 채비 사진 연합뉴스 금융권이 신규 고객 유치의 수단으로 여러 사람이 함께 계좌를 관리할 수 있는 모임통장을 잇달아 내놓고 있다 모임통장은 출시관리에 큰 비용이 들지 않지만 많은 자금을 조달할 수 있어 효자 상품이다 차별화를 위해 단체적금 영수증 첨부 기능 등을 내놓는 곳도 등장했다 일 금융권에 따르면 신한은행은 다음달 SOL모임통장을 출시한다 신한은행은 년 모임통장 전용 앱 김총무를 출시했지만 저조한 이용률 등을 이유로 년 월 서비스를 중단했다가 년 만에 재출시한다 시중은행 모임통장 상품 중 최초로 적금통장 기능을 갖췄다 모임 멤버들이 특정 목표를 위해 금액과 기간을 설정하고 다 같이 돈을 모을 수 있도록 했다 신한은행 고객이 아니더라도 모바일 웹을 통해 모임통장에 참여할 수 있다 과거 금융권에서 출시된 모임통장은 해당 금융사 이용자가 앱을 설치해야만 참여가 가능했다 거래내역 영수증 첨부 기능도 더해 통장에 들어오거나 나가는 돈의 액수와 사용처를 투명하게 관리할 수 있도록 했다 참여 멤버들에게 모임통장의 거래 세부 내용을 속이는 방식으로 곗돈 먹튀가 발생하는 일을 막을 수 있다는 게 신한은행 측 설명이다 모임통장에 모인 돈을 한동안 쓸 일이 없을 때 파킹통장에 넣어둬 이자를 더 받는 것도 가능하다 KB국민은행도 모임통장 서비스를 운영 중이다 지난해 하반기 선보인 KB모임금고라는 파킹통장 서비스다 모임통장 서비스에 가입한 고객이 여유자금을 보관할 수 있도록 하는 상품이며 최고금리는 연 다 우리은행은 지난해 월 새 뱅킹 앱인 뉴원뱅킹 출시와 함께 앱에 모임통장 기능을 신설했다 거래 내역은 그대로 두고 모임장을 교체할 수 있으며 하나의 모임에서 카드 여러 장을 발급받을 수 있도록 했다 하나은행에서도 하나원큐 앱을 통해 모임통장 서비스 이용이 가능하다 기존에 사용 중이던 통장에 모임 기능을 연결해 모임통장으로 쓸 수 있다 납부일미납사실 자동 알림 기능을 갖췄으며 음식점이나 커피전문점 등에서 모임 전용 체크카드 이용 시 월 최대 만원의 캐시백을 받을 수 있다 NH농협은행 또한 올원뱅크 앱과 연계한 NH모여라통장을 운영 중이다 모임통장은 수시 입출금통장 형태가 대부분이다 기본금리는 연 수준이다 카카오뱅크 케이뱅크 토스뱅크와 같은 인터넷전문은행도 다양한 상품을 내놓고 있다 저축은행도 조만간 모임통장 서비스에 뛰어들 예정이다 저축은행중앙회는 모임통장 시스템 구축 절차를 밟고 있다 시중은행 관계자는 은행은 모임통장으로 일반 예적금 상품보다 많은 자금을 조달할 수 있다면서 모임에 참여하는 사람들이 은행의 다른 서비스를 이용하도록 유도할 수 있다는 것도 장점이라고 설명했다
1,이코노미 플러스지난해 불법사금융 피해신고 급증채권추심 년만에 배로,지난해 불법사금융 피해신고 급증채권추심 년만에 배로 사진연합뉴스 사진연합뉴스 지난해 월까지 불법 사금융 피해 상담신고 건수가 최근 년새 가장 많은 수준으로 급증한 것으로 집계됐다 추심방식이 악랄해지면서 채권추심과 관련한 피해상담 건수는 년만에 배 수준으로 폭증했다 일 금융감독원이 국회 정무위원회 소속 서범수 국민의힘 의원에게 제출한 자료를 보면 지난해 월 금감원 불법 사금융 피해신고센터에 접수된 상담신고 건수는 만건으로 전년 같은 기간 만건에 비해 늘었다 같은 기간 기준 년 건 년 건 년 건에 비해서 큰 폭으로 증가해 최근 년새 가장 많았다 불법 사금융 피해 상담신고 건수는 연간 기준으로도 년 건 년 건 년 만건 년 만건 등으로 급증세를 이어가고 있다 지난해 월까지 세부 피해 유형을 보면 미등록 불법 대부업체 관련이 건으로 가장 많았고 채권추심 건 고금리 건 불법광고 건 불법 수수료 건 유사수신 건 등으로 뒤를 이었다 특히 채권추심 관련 피해 상담신고 건수는 전년 같은 기간 건에 비해 급증해 채권추심 피해가 심화하고 있다는 점을 드러냈다 채권추심 피해상담신고 건수는 같은 기간 기준 년 건 년 건 년 건 년 건 년 건 등으로 급증세를 이어가고 있다 년만에 배로 폭증한 셈이다 채권추심 방식은 지독해지고 있다 지난해 월에는 유치원생 딸을 홀로 키우던 대 싱글맘이 딸이 다니는 유치원 선생님에게까지 빚 독촉을 하는 불법추심에 시달린 끝에 스스로 목숨을 끊었다 서울 북부지검은 지난 일 고리로 돈을 빌려준 뒤 상환을 독촉하면서 지속적으로 협박해 결국 죽음으로 내몬 사채업자인 대 A씨를 구속기소했다 그는 지난해 월 대부업 등록을 하지 않은 채 명에게 합계 만원을 법정이자율을 배도 훌쩍 넘어서는 고리로 빌려준 뒤 이들의 가족과 지인에게 협박성 메시지를 전송하는 등 불법적 추심행위를 한 혐의를 받는다 피해상담신고 건수는 급증하고 있지만 금감원의 수사 의뢰 건수는 지난해 월까지 건으로 년 같은 기간 건 년 건 년 건에 비해 줄어든 것으로 집계됐다 서범수 의원은 불법사금융과 보이스피싱은 특히 서민과 취약계층의 자금 수요가 몰리는 설 전후로 굉장히 기승을 부리는 만큼 각별히 주의해야 할 필요가 있다면서 정부가 불법사금융 관련 수사 인력을 보강하고 명절 전후 기간을 불법사금융 특별근절 기간으로 정해 정기적으로 운영하는 방안을 고려할 필요가 있다고 밝혔다 경기 악화로 서민과 취약계층의 어려움이 고조되면서 생계형 자금 수요가 지속되고 있지만 대부업체를 비롯한 제도권 금융의 대출 문턱은 높아지면서 불법사금융으로 내몰리는 이들은 늘어나고 있다 지난해 상반기 대형 대부업체의 대출잔액은 조억원으로 전년 말보다 인 억원 감소했다 대부업체 이용자는 만명으로 같은 기간 만천명 줄었다 상반기 대형 대부업체의 연체율은 까지 치솟아 역대 최고 수준을 경신했다 서민금융연구원이 대부업불법사금융 이용자 천명을 상대로 지난해 월 벌인 설문조사 결과를 보면 전년 대부업체에 대출을 신청해 거절당한 이들의 비율은 로 년에 비해 큰 폭으로 늘어났다 년 개인신용평점 하위 를 대상으로 불법사금융으로 이동 규모를 추정한 결과 만만명으로 추정돼 년에 비해 최소 명 최대 만명이 증가한 것으로 보인다고 연구원은 설명했다
2,이코노미 플러스 금융당국 저축은행 여러곳 취약 추가통보적기시정조치 가능성,금융당국 저축은행 여러곳 취약 추가통보적기시정조치 가능성 자산건전성 악화 잇따라업계 강제 구조조정 압력 커져 사진연합뉴스 금융당국이 저축은행 여러 곳의 자산건전성 지표에 취약 등급을 추가 확정하면서 적기시정조치 부과 논의도 이어질 것으로 전망된다 지난달 금융위원회가 안국저축은행과 라온저축은행에 적기시정조치를 부과한 데 이어 강제 구조조정 규모가 확대될지 주목된다 지난 일 금융당국과 저축은행업권에 따르면 금융감독원은 최근 금융위원회에 작년 월 말 기준 자산건전성 지표와 관련해 저축은행 곳을 대상으로 한 경영실태평가의 최종 평가 등급을 전달했다 이들 대부분이 등급취약 등급을 받은 것으로 알려졌다 금감원은 작년 월 말 기준으로도 곳에 취약 등급을 확정했으며 금융위는 이를 토대로 경영개선계획서를 받아본 뒤 안국라온저축은행 등 곳에 적기시정조치를 내렸다 저축은행업감독규정에 따르면 경영실태평가 종합평가등급이 등급이거나 자산건전성 또는 자본 적정성 평가등급이 등급 이하면 금융당국의 적기시정조치 대상이 될 수 있다 적기시정조치는 금융당국이 부실 위험 금융사에 내리는 경영개선 조치권고요구명령로 최고 단계인 명령에선 영업이 정지되거나 합병매각될 수 있다 금융당국 관계자는 자본 적정성에 문제 있는 저축은행은 없지만 자산건전성이 좋지 못한 상황이라고 설명했다 업권에서는 이번에도 작년 월 말 기준 경영실태평가 때처럼 가장 낮은 단계의 조치가 부과되거나 지표 개선 등을 감안해 조치가 유예될 것으로 예상하고 있다 그러나 부동산 프로젝트파이낸싱PF 구조조정이 계속 이어지는 가운데 건설 경기 부진 등이 이어지면서 저축은행업계 구조조정 압력은 계속 커질 것으로 보인다 아울러 금감원은 작년 월 말 기준 저축은행 대상 경영실태평가 결과도 정리해 금융위에 추가 통보할 계획이다 올해 적기시정조치를 부과받는 저축은행이 추가로 쌓일 수 있다는 의미다 지난달 적기시정조치 부과를 받은 라온저축은행은 코스닥 상장사인 베셀에 매각을 추진 중이며 안국

In [32]:
# 데이터 준비  : 뉴스 제목과 내용을 하나의 텍스트 데이터로 통합
data["통합 텍스트"] = data["뉴스 제목 정제"] + " " + data["뉴스 내용 정제"]

data["통합 텍스트"] = data["통합 텍스트"].fillna("").astype(str)

In [36]:
from konlpy.tag import Mecab  # 형태소 분석기
from sklearn.feature_extraction.text import TfidfVectorizer # TF-IDF 계산 도구

# 1. 형태소 분석기로 명사 추출
# 형태소 분석기 초기화
mecab = Mecab()

# 텍스트 데이터를 형태소 분석 후 명사 추출
data["형태소 분석 텍스트"] = data["통합 텍스트"].apply(
    lambda x: " ".join(mecab.nouns(x))
)

# 3. TF-IDF 계산
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(data["형태소 분석 텍스트"])

# 4. 키워드 추출

# 4.1 각 문서별 주요 키워드 출력
terms = vectorizer.get_feature_names_out()
tfidf_sum = tfidf_matrix.sum(axis=0).A1  # 각 단어의 TF-IDF 점수 합산
sorted_indices = tfidf_sum.argsort()[::-1]  # 점수 기준 정렬

print("전체 데이터 주요 키워드:")
for idx in sorted_indices[:10]:  # 상위 10개 키워드 출력
    print(f"{terms[idx]}: {tfidf_sum[idx]}")

전체 데이터 주요 키워드:
트럼프: 199.73635499618973
서울: 177.47869507373724
시장: 171.75870895712984
미국: 162.44197213916405
기업: 148.17511104525246
은행: 144.31455647383547
달러: 134.39383562367058
금융: 133.15088671710373
사업: 128.15304065534445
투자: 115.89292682638136
